# Plot master STFs

We plot the master figure of STF associated with the dynamic rupture simulation.

2024.06.17 Kurama Okubo

- 2024.08.01 update to compute the half maximum amplitude width
- 2024.09.04 **Major update:** compute HMPW and plot the master figures for STFs.
- 2024.09.14 update for the total length of zeropadded STF
- 2025.1.31 Clean up the notebook for the master plot.
- 2025.2.20 Update the STF time shift to be aligned to the snapshots
- 2025.4.26 update with v65 datasets.

In [ ]:
import os
import shutil
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.cm import ScalarMappable
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import warnings
import time
from datetime import datetime

import pickle

from scipy.optimize import minimize

from scipy.signal import freqz
from scipy import signal
from scipy import integrate

import h5py

import seaborn as sns

from post_dynrup_func import *

%load_ext autoreload
%autoreload 2

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 11
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = False

plt.rcParams['axes.linewidth'] = 0.75

In [ ]:
figdir = "../figure/05_master_STF"
if not os.path.exists(figdir):
    os.makedirs(figdir)

# Read STF of dynamic rupture model

In [ ]:
a_patch = 4.0e-3
a_nuc = 2.5e-3
a_margin = 4.08e-3

rupturetype = "pulse"
pdcscaling = 0.6 #0.65
bgbeta= 0.35 #0.4

sig_n = 6e6

IfBinaryOutput = True

nb_x_elements = 1024 #128 #128 #128

casestr = f"a={a_patch*1e3:.2f}_ruptype={rupturetype}_pdcscaling={pdcscaling:.3f}_sn={sig_n/1e6:.1f}MPa_hatfr=0.3_bgbeta={bgbeta:.2f}"

Ifmasteroutput=True # add file key to the master output
if Ifmasteroutput:
    filekey = "_master"
else:
    filekey = ""

In [ ]:
finame=f"../../preprocess_modelsetup/data/gouge_dynamicrupture_modelparam_{casestr}{filekey}.csv"

# Read model parameters
df_modelparam = pd.read_csv(finame, index_col=0)

In [ ]:
# read the STF processed in the previous notebook
fo_dynrup = h5py.File(f"../data/01_M0andSTF/DynrupModelData_{casestr}_{nb_x_elements}{filekey}.hdf5", 'r+')

tvec_dynrup = np.array(fo_dynrup["param/tvec_dynrup"])

In [ ]:
# read STF best-fit associated with the dynamic rupture model
df_dynrupfitparam = pd.read_csv(f"../data/dynrup_bestfit_sourceparam_{casestr}{filekey}.csv", index_col=None)
df_dynrupfitparam

In [ ]:
# load color dictionary consistent to the plot of the repeated waveforms
repeated_sensor_lcdict = "OL08" # the color dict is same for all the sensor although separately saved.
gougepatch_id = "G3"
with open(f'../../../../ComputeScaling/data/01_plot_gougeevents/lc_dict_{gougepatch_id}_{repeated_sensor_lcdict}.pkl', 'rb') as fi:
    lc_dict = pickle.load(fi)

In [ ]:
# lc_dict

# Plot the STF

For the clarity of the evolution of dynamic rupture model, we do not zero-pad the STF associated with the dynamic rupture model.
We updated to shift the time using the onset of M129 to synchronize it to the snapshots.

In [ ]:
fo_dynrup.keys()

In [ ]:
tvec_dynrup = np.array(fo_dynrup["param/tvec_dynrup"])
dt_dynrup = tvec_dynrup[1] - tvec_dynrup[0]

In [ ]:
# tvec_dynrup

In [ ]:
zerowin_pre = 3e-6
zerowin_post = 3e-6

In [ ]:
expr_id = 87
model_ids = [24, 50, 52, 72, 129]

In [ ]:
t_plot_list = np.loadtxt(f"../data/t_plot_list_{casestr}{filekey}.txt", delimiter=',')
t_plot_list

In [ ]:
model_id_ref = 129 # align with this event for dynamic rupture model
df_dynrupfitparam_select_ref = df_dynrupfitparam[df_dynrupfitparam["event_id"]==model_id_ref]
Tshift_bestfit_ref = df_dynrupfitparam_select_ref["Tshift_bestfit"].values[0] # This is the onset of best-fit STF on the dynrup
Tshift_bestfit_ref

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.9, 5.4))

# plot normalized STF in the inset
ax_inset = fig.add_axes([0.62, .6, .25, .25])

snapshot_gougeevent_id = 129

for ii, model_id in enumerate(model_ids):

    datacase = f"fb03-087__{model_id:04d}"

    #-----------------#    
    # plot synthetic
    #-----------------#

    df_modelparam_selected = df_modelparam[df_modelparam.index == model_id] # best-fit source param from observation
    M0_mean = df_modelparam_selected["M0_mean"].values[0]
    Tw_mean = df_modelparam_selected["Tw_mean"].values[0]
       
    # compute synthetic STF
    tvec_syn = np.linspace(0, Tw_mean, int(Tw_mean/dt_dynrup))
    dt_tvec_syn = tvec_syn[1]-tvec_syn[0]
    STF_syn = stf_cosine(tvec_syn, Tw_mean, M0_mean)
    # STF_syn = stf_kupper(tvec_syn, Tw_mean, M0_mean)

    # zero-padding
    tpre_syn = -np.arange(dt_tvec_syn, zerowin_pre, step=dt_tvec_syn)[::-1]
    tpost_syn = np.arange(tvec_syn[-1]+dt_tvec_syn, tvec_syn[-1]+zerowin_post, step=dt_tvec_syn)
    tvec_syn_padded = np.hstack([tpre_syn, tvec_syn, tpost_syn])
    
    post_add = 0
    if np.mod(len(tvec_syn_padded), 2) == 1: # make the length to be even
        # make tvec length as even
        tvec_syn_padded = np.hstack([tvec_syn_padded, tvec_syn_padded[-1]+dt_tvec_syn])
        post_add = 1
    
    STF_rec_padded = np.hstack([np.zeros(len(tpre_syn)), STF_syn, np.zeros(len(tpost_syn)+post_add)])

    if model_id == snapshot_gougeevent_id:
        labelsyn="Cosine STF based on observed\nsource parameters"
    else:
        labelsyn=None
        
    ax.plot((tvec_syn_padded + Tshift_bestfit_ref)*1e6, STF_rec_padded/1e6, ls=":", c=lc_dict[datacase], lw=1.5, label=labelsyn) # before alignment


    #-----------------#    
    # plot dynamic rupture model
    #-----------------#    

    df_dynrupfitparam_select = df_dynrupfitparam[df_dynrupfitparam["event_id"]==model_id]
    Tshift_bestfit = df_dynrupfitparam_select["Tshift_bestfit"].values[0] # This is the onset of best-fit STF on the dynrup

     # Plot dynamic rupture model
    df_modelparam_selected = df_modelparam[df_modelparam.index == model_id]
    simulation_name = f"fb03-{expr_id:03d}__{df_modelparam_selected.index[0]:04d}_{casestr}"
    key_STF = f"STF_rec_{simulation_name}"
    STF_rec = np.array(fo_dynrup[f'dynrup_{model_id:04d}/STF_rec'])
    STF_max = np.max(STF_rec)
    STF_maxarg = np.argmax(STF_rec)

    if model_id == snapshot_gougeevent_id:
        labeldynrup="Dynamic rupture model"
    else:
        labeldynrup=None
        
    ax.plot((tvec_dynrup - Tshift_bestfit + Tshift_bestfit_ref)*1e6, STF_rec/1e6, "-", c=lc_dict[datacase], lw=1.5, zorder=3, label=labeldynrup)
    # ax.plot((tvec_dynrup - 0 + Tshift_bestfit_ref)*1e6, STF_rec/1e6, "-", c=lc_dict[datacase], lw=1.5, zorder=3, label=labeldynrup)
    
    # Annotate snapshot timing
    if (model_id==snapshot_gougeevent_id) & (rupturetype=="pulse"):
        t_snapshots_dynrup = [Tshift_bestfit] # snap timing list after time shifted of the plot

        for t_snap in t_plot_list:
            t_snap_ind = np.where(tvec_dynrup >= t_snap)[0][0]
    
            # compute t_snap associated with the dynamic rupture model
            t_snapshots_dynrup.append(tvec_dynrup[t_snap_ind]-Tshift_bestfit)
            
            ax.plot((tvec_dynrup[t_snap_ind]-Tshift_bestfit + Tshift_bestfit_ref)*1e6, STF_rec[t_snap_ind]/1e6, "s",
                    mfc="w", mec="k", ms=5., zorder=5)

    
    #-----------------#    
    # debug: plot best-fit STF of dynamic rupture model
    #-----------------# 
    M0_bestfit = df_dynrupfitparam_select["M0_bestfit"].values[0]
    Tw_bestfit = df_dynrupfitparam_select["Tw_bestfit"].values[0]
       
    # compute synthetic STF
    tvec_dynrupbestfit = np.linspace(0, Tw_bestfit, int(Tw_mean/dt_dynrup))
    dt_dynrupbestfit = tvec_dynrupbestfit[1]-tvec_dynrupbestfit[0]
    STF_dynrupbestfit = stf_cosine(tvec_dynrupbestfit, Tw_bestfit, M0_bestfit)
    # ax.plot((tvec_dynrupbestfit)*1e6, STF_dynrupbestfit/1e6, ls="-.", c=lc_dict[datacase], lw=2.0) # before alignment

    #-----------------#    
    # plot normalized STF
    #-----------------#

    ax_inset.plot((tvec_dynrup - Tshift_bestfit + Tshift_bestfit_ref)*1e6, STF_rec/STF_max, "-", c=lc_dict[datacase], lw=1.5)


# Annotate the event IDs

annot_xfactor = [0.03, 0.02, 0.02, 0.02, 0.0]
annot_yfactor = [0.95, 0.97, 0.97, 1.0, 0.97]
annot_edge_t = [0.8, 0.85, 0.9, 0.93, 1]
annot_locs = [(-0.8, 0.1), (-0.8, 0.15), (-0.8, 0.25), (-0.8, 0.27), (-0.8, 0.3)]

for ii, model_id in enumerate(model_ids):

    df_dynrupfitparam_select = df_dynrupfitparam[df_dynrupfitparam["event_id"]==model_id]
    Tshift_bestfit = df_dynrupfitparam_select["Tshift_bestfit"].values[0] # This is the onset of best-fit STF on the dynrup
    df_modelparam_selected = df_modelparam[df_modelparam.index == model_id]
    simulation_name = f"fb03-{expr_id:03d}__{df_modelparam_selected.index[0]:04d}_{casestr}"
    key_STF = f"STF_rec_{simulation_name}"
    STF_rec = np.array(fo_dynrup[f'dynrup_{model_id:04d}/STF_rec'])
    # ax.plot((tvec_dynrup - Tshift_bestfit + Tshift_bestfit_ref)*1e6, STF_rec/1e6, "-", c=lc_dict[datacase], lw=1.5, zorder=3, label=labeldynrup)

    (a_xy, b_xy) = annot_locs[ii]
    xy_annot_tind = np.where((tvec_dynrup - Tshift_bestfit + Tshift_bestfit_ref)*1e6 >= annot_edge_t[ii])[0][0]
    xy_annot = ((1+annot_xfactor[ii])*annot_edge_t[ii], annot_yfactor[ii]*STF_rec[xy_annot_tind]/1e6)
    xytext = (annot_edge_t[ii]+ a_xy, STF_rec[xy_annot_tind]/1e6 + b_xy)

    ax.annotate(f"M{model_id:g}", xy=xy_annot, xytext=xytext,
        arrowprops=dict(arrowstyle = '-', connectionstyle = 'arc3', lw=0.75), 
        horizontalalignment="left", zorder=4, fontsize=11., c="k")
    



        
ax.set_xlabel("Time [μs]")
ax.set_ylabel(r"$\dot{M}_0(t)$ [MNm/s]")

ax.set_xlim([-1.4, 4.4])
# ax.set_xlim([0, 10])
ax.set_ylim([-0.05, 1.15])

ax.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, zorder=-1)
ax.set_axisbelow('True')

#https://stackoverflow.com/a/34576778
# ax.legend(loc=2)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='upper left', frameon=False)

# inset
ax_inset.set_xlim([-0.5, 3.5])
ax_inset.set_ylim([-0.1, 1.1])
ax_inset.set_yticks([0, 1])
# ax_inset.set_xlabel("Time [μs]")
ax_inset.grid(True, c=np.array([230, 230, 230])/255, lw=0.25, zorder=-1)
ax_inset.set_axisbelow('True')

# plt.suptitle(f'Q{Qinv_quart} water-level={k_waterlevel:.2f}', y=0.98)
# plt.tight_layout()

plt.savefig(figdir + f"/master_STFcomparison_{casestr}_{nb_x_elements}{filekey}.png", dpi=300, bbox_inches="tight")
plt.savefig(figdir + f"/master_STFcomparison_{casestr}_{nb_x_elements}{filekey}.eps", bbox_inches="tight")


In [ ]:
xy_annot

In [ ]:
t_plot_list

In [ ]:
Tshift_bestfit

In [ ]:
dt_dynrup